<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 09:48:34
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 30.60 K (0.22%)
Current PnL: -12.84 L (-8.94%)
CY Booked + Current PnL: -5.88 L (-4.1%)
-------------------
Total profit:  4.92 L
Total loss:  -17.76 L
-------------------
Total Booked + Current PnL: 21.10 L (18.11%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.35 L (51.97%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.52,H-LC,91.67,113805.0,10239.0,10470.0,-0.38,9.89,9.20,20.00,32.0,0.98,0.83,25.35,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.08,H-LC,87.50,111272.0,6960.0,10571.0,0.48,6.67,9.50,16.81,27.0,0.66,0.81,23.79,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,0.15,M-LC,28.12,173722.0,32882.0,11605.0,0.41,23.35,6.68,31.59,88.0,2.83,1.27,50.90,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-213.31,H-SC,98.96,81754.0,-13562.0,13604.0,1.07,-14.23,16.64,0.04,157.0,-1.00,0.60,61.25,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,29.17,214650.0,13170.0,14038.0,0.39,6.54,6.54,13.51,77.0,0.94,1.56,19.96,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,8.33,181100.0,1292.0,111123.0,0.03,0.72,61.36,62.52,173.0,0.01,1.32,35.05,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,30.21,150560.0,-353.0,60811.0,-0.12,-0.23,40.39,40.06,103.0,-0.01,1.10,23.15,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,50.00,100451.0,-53.0,30688.0,0.51,-0.05,30.55,30.49,97.0,-0.00,0.73,32.33,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,TRIDENT,37.35,48.00,29.34,M-SC,84.38,81263.0,-10992.0,37300.0,5.72,-11.91,45.90,28.51,233.0,-0.29,0.59,39.05,XR,NTT,TEXTILES
16,BLUSPRING,226.45,84.93,NaN,NaN,46.88,18600.0,-30993.0,0.0,3.99,-62.49,0.00,-62.50,270.0,-inf,0.14,24.73,XY24,ATH,MISC
13,BATAINDIA,1550.24,2096.00,-29.87,M-SC,73.96,104962.0,-23708.0,69002.0,2.96,-18.43,65.74,35.20,205.0,-0.34,0.77,7.09,X40,NTT,FOOTWEAR
32,ENRIN,1377.95,3079.70,NaN,NaN,95.83,76992.0,42543.0,0.0,2.65,123.50,0.00,123.50,274.0,inf,0.56,17.73,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,175542.0,33700.0,22873.0,2.34,23.76,13.03,39.88,262.0,1.47,1.28,77.90,XY24,NTT,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,RELAXO,902.64,1176.00,-30.57,H-SC,92.71,62784.0,-52754.0,87747.0,-3.13,-45.66,139.76,30.28,131.0,-0.60,0.46,22.66,X40N,NTT,FOOTWEAR
3,ALKYLAMINE,2347.98,4546.37,9.28,X-SC,97.92,101781.0,818.0,93710.0,-1.29,0.81,92.07,93.63,67.0,0.01,0.74,51.60,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,61.46,64608.0,-48941.0,88991.0,-1.26,-43.10,137.74,35.27,265.0,-0.55,0.47,58.13,XR,NTT,HOTELS
22,CIPLA,1495.00,1795.00,-22.16,H-LC,41.67,100165.0,0.0,20103.0,-1.03,0.00,20.07,20.07,30.0,0.00,0.73,8.90,X40N,BTT,PHARMA
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180860.0,16190.0,52902.0,-0.99,9.83,29.25,41.95,124.0,0.31,1.32,18.03,X40N,ATH,AC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,23.96,127710.0,-14769.0,15389.0,0.64,-10.37,12.05,0.44,247.0,-0.96,0.93,50.73,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,25.00,151187.0,-19954.0,19896.0,0.23,-11.66,13.16,-0.03,191.0,-1.00,1.10,8.66,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.31,H-SC,98.96,81754.0,-13562.0,13604.0,1.07,-14.23,16.64,0.04,157.0,-1.00,0.60,61.25,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,94.79,149321.0,-25238.0,25265.0,0.28,-14.46,16.92,0.02,133.0,-1.00,1.09,29.48,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-67.65,H-SC,44.79,225810.0,-43857.0,80185.0,0.46,-16.26,35.51,13.47,137.0,-0.55,1.65,13.02,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,36.46,265582.0,-30352.0,124266.0,-0.20,-10.26,46.79,31.73,1.0,-0.24,1.94,6.30,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,68.75,290875.0,29274.0,116350.0,-0.12,11.19,40.00,55.67,2.0,0.25,2.12,17.98,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,54.17,269942.0,-405.0,35038.0,-0.01,-0.15,12.98,12.81,5.0,-0.01,1.97,12.90,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,18.49,X-LC,71.88,271515.0,42282.0,25007.0,0.31,18.44,9.21,29.35,10.0,1.69,1.98,29.01,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-1.77,X-LC,27.08,83100.0,-27072.0,116149.0,1.40,-24.57,139.77,80.85,11.0,-0.23,0.61,22.39,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3079.70,NaN,NaN,95.83,76992.0,42543.0,0.0,2.65,123.50,0.00,123.50,274.0,inf,0.56,17.73,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,37.50,50698.0,9443.0,1364.0,1.31,22.89,2.69,26.19,272.0,6.92,0.37,19.36,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,175542.0,33700.0,22873.0,2.34,23.76,13.03,39.88,262.0,1.47,1.28,77.90,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,0.15,M-LC,28.12,173722.0,32882.0,11605.0,0.41,23.35,6.68,31.59,88.0,2.83,1.27,50.90,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,45.83,231643.0,6681.0,160969.0,0.79,2.97,69.49,74.53,189.0,0.04,1.69,34.49,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.64,25.90,H-SC,34.90,128646.0,3349.0,55729.0,-0.48,2.67,43.32,47.15,143.0,0.06,0.94,60.94,MH,ATH,POWER
20,CAMS,3643.00,5250.99,2.76,H-SC,21.88,115500.0,13496.0,31532.0,-0.41,13.23,27.30,44.14,126.0,0.43,0.84,31.62,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,28.12,173722.0,32882.0,11605.0,0.41,23.35,6.68,31.59,88.0,2.83,1.27,50.90,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-2.86,M-LC,86.46,166346.0,15401.0,94052.0,0.21,10.20,56.54,72.51,71.0,0.16,1.21,14.58,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3079.70,NaN,NaN,95.83,76992.0,42543.0,0.0,2.65,123.50,0.00,123.50,274.0,inf,0.56,17.73,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.43,M-SC,60.42,202978.0,31357.0,130718.0,0.58,18.27,64.40,94.43,201.0,0.24,1.48,49.81,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,86.46,166346.0,15401.0,94052.0,0.21,10.20,56.54,72.51,71.0,0.16,1.21,14.58,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-47.70,H-SC,75.00,136120.0,12784.0,124182.0,1.18,10.37,91.23,111.05,123.0,0.10,0.99,33.37,AR,ATH,MISC
54,KPIGREEN,497.21,731.64,25.90,H-SC,34.90,128646.0,3349.0,55729.0,-0.48,2.67,43.32,47.15,143.0,0.06,0.94,60.94,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.93,NaN,NaN,46.88,18600.0,-30993.0,0.0,3.99,-62.49,0.00,-62.50,270.0,-inf,0.14,24.73,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,61.46,64608.0,-48941.0,88991.0,-1.26,-43.10,137.74,35.27,265.0,-0.55,0.47,58.13,XR,NTT,HOTELS
32,ENRIN,1377.95,3079.70,NaN,NaN,95.83,76992.0,42543.0,0.0,2.65,123.50,0.00,123.50,274.0,inf,0.56,17.73,AR,ATH,ELECTRICAL
85,TRIDENT,37.35,48.00,29.34,M-SC,84.38,81263.0,-10992.0,37300.0,5.72,-11.91,45.90,28.51,233.0,-0.29,0.59,39.05,XR,NTT,TEXTILES
58,MASFIN,326.60,399.50,-17.80,H-SC,65.62,93210.0,-4770.0,26639.0,0.50,-4.87,28.58,22.32,145.0,-0.18,0.68,35.19,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-12.60,L-SC,100.00,175542.0,33700.0,22873.0,2.34,23.76,13.03,39.88,262.0,1.47,1.28,77.90,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-213.31,H-SC,98.96,81754.0,-13562.0,13604.0,1.07,-14.23,16.64,0.04,157.0,-1.00,0.60,61.25,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,9.28,X-SC,97.92,101781.0,818.0,93710.0,-1.29,0.81,92.07,93.63,67.0,0.01,0.74,51.60,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.59,H-SC,96.88,180860.0,16190.0,52902.0,-0.99,9.83,29.25,41.95,124.0,0.31,1.32,18.03,X40N,ATH,AC
32,ENRIN,1377.95,3079.70,NaN,NaN,95.83,76992.0,42543.0,0.0,2.65,123.50,0.00,123.50,274.0,inf,0.56,17.73,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.85
2,50,70.57


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.54
LC    33.92
MC    21.48
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      12.91
XY25     12.05
X40N     11.93
XR       10.04
OX40N     8.20
AR        8.20
MH        1.67
X5K       1.45
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.48
M-SC    15.95
H-LC    14.08
X-LC    12.93
H-MC     9.50
X-MC     7.57
M-LC     5.87
X-SC     4.43
M-MC     4.05
L-SC     2.68
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-0.22,33.80
IT,12.05,-9.79,63.71
FINANCE,8.51,-11.72,52.25
BANKS,7.72,-7.58,51.47
MISC,5.85,-20.01,76.26
PAINTS,5.70,-11.75,33.78
ELECTRICAL,5.47,1.91,44.25
HEALTHCARE,4.66,-1.03,30.55
AUTO,4.31,-14.40,64.70


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2646000.0,26
XR,1080047.0,13
AR,1024418.0,10
X40,684821.0,10
X40N,527921.0,11
XY25,467251.0,8
OX40N,305588.0,11
SR,175972.0,2
MH,86417.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1976398.0,20
M-SC,1754824.0,21
H-LC,607874.0,12
X-LC,537874.0,8
H-MC,531014.0,8
X-MC,414674.0,6
M-MC,413943.0,3
X-SC,310181.0,5
M-LC,255593.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      883876.0      6
M-SC       XY24      806562.0      7
H-SC       AR        526353.0      3
M-SC       XR        331020.0      4
M-MC       XY24      302820.0      2
H-MC       XY24      299307.0      3
X-LC       X40       276999.0      3
H-SC       XR        217504.0      3
           X40N      207118.0      4
H-LC       X40N      193733.0      4
M-SC       AR        183507.0      2
           OX40N     161718.0      5
X-MC       X40       155365.0      2
H-LC       X40       155252.0      3
L-SC       XR        150230.0      2
M-LC       XY24      147503.0      2
X-SC       XR        122622.0      1
M-SC       XY25      120753.0      1
X-LC       AR        116149.0      1
M-MC       XR        111123.0      1
X-MC       XY25      102987.0      1
X-LC       XY25       98361.0      3
M-LC       XR         94052.0      1
X-SC       SR         93710.0      1
X-MC       XY24       86938.0      1
H-SC       OX40N      85818.0      3
H-MC       XY25       83783.0      1
M-SC       SR         82262.0      1
H-LC       AR         80189.0      1
M-SC       X40        69002.0      1
H-LC       X5K        68974.0      2
           X200       67667.0      1
H-MC       AR         60811.0      1
L-SC       AR         57409.0      1
H-SC       MH         55729.0      1
L-MC       XR         53496.0      1
X-SC       XY24       52698.0      1
X-MC       X40N       52483.0      1
L-LC       XY25       47329.0      1
X-LC       X40N       46365.0      1
H-LC       XY24       42059.0      1
X-SC       OX40N      41151.0      2
H-MC       MH         30688.0      1
           X40N       28222.0      1
           X40        28203.0      1
L-SC       XY24       22873.0      1
X-MC       OX40N      16901.0      1
M-LC       XY25       14038.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
